# ArchiCrop tutorial - Sole crop

This notebook is a tutorial for executing ArchiCrop model for a sole crop.

## 0. Imports

In [1]:
import numpy as np
from oawidgets.plantgl import *
from openalea.plantgl.all import Material, Color3, Scene
import oawidgets.mtg

from openalea.archicrop.archicrop import ArchiCrop
from openalea.archicrop.display import build_scene, display_scene
%gui qt

## 1. Set management parameters

Set parameters regarding the spatial (and temporal) configuration of the crop.

In [2]:
sowing_density = 10
inter_row = 0.4

## 2. Format crop-scale growth and senescence dynamics

From crop-scale data either measured or simulated with a crop model, generate a dictionnary of dictionnaries for each day with the following values :
 - "Thermal time" (float): cumulated thermal time from beginning of simulation to current day (in °C.day)
 - "Phenology" (str): current phenological stage (germination, juvenile, exponential or repro)
 - "Plant leaf area" (float): plant leaf area (in cm²) 
 - "Leaf area increment" (float): leaf area increment compared to previous day (in cm²) 
 - "Plant senescent leaf area" (float): senescent plant leaf area (in cm²) 
 - "Senescent leaf area increment" (float): senescent leaf area increment compared to previous day (in cm²) 
 - "Plant height" (float): plant height (in cm) 
 - "Height increment" (float): height increment compared to previous day (in cm).

In [3]:
# Time series 
# for thermal time, plant leaf area, plant senescent leaf area and plant height
# for the example, one value every 18 days
thermal_time = np.cumsum([tt*18 for tt in [3, 10, 11, 13, 18, 14, 13, 15, 13, 10]]) # cumulated thermal time
lai = [0.01, 0.77, 2.46, 5.00, 5.81, 5.81, 5.81, 5.81, 5.81, 5.81] # crop-scale

# To remove
lai_scale = 10
lai = [la*lai_scale for la in lai]

leaf_area_plant = [la*1e4/sowing_density for la in lai] # for 1 plant
sen_lai = [0.00, 0.00, 0.00, 0.20, 0.60, 1.00, 2.00, 3.00, 4.00, 5.81] # crop-scale
sen_leaf_area_plant = [sen_la*1e4/sowing_density for sen_la in sen_lai] # for 1 plant
height_canopy = [h*100 for h in [0.00004, 0.02, 1.04, 2.05, 2.42, 2.46, 2.46, 2.46, 2.46, 2.46]] # height of the canopy

# Phenology parameters, in days
emergence = 0 # start of the simulation for a sole crop
end_juv = 3 
max_lai = 6

# Fill dict of dicts for daily dynamics
daily_dynamics = {
        i+1: {"Thermal time": thermal_time[i],
              "Phenology": 'germination' if i+1 < emergence else 'juvenile' if emergence <= i+1 < end_juv else 'exponential' if end_juv <= i+1 < max_lai else 'repro',
              "Plant leaf area": leaf_area_plant[i], 
              "Leaf area increment": leaf_area_plant[i] - leaf_area_plant[i-1] if i > 0 else leaf_area_plant[i], 
              "Plant senescent leaf area": sen_leaf_area_plant[i],
              "Senescent leaf area increment": sen_leaf_area_plant[i] - sen_leaf_area_plant[i-1] if i > 0 else sen_leaf_area_plant[i],
              "Plant height": height_canopy[i], 
              "Height increment": height_canopy[i] - height_canopy[i-1] if i > 0 else height_canopy[i]
             }
        for i in range(len(thermal_time))
}

## 3. Set plant architectural parameters

Set topological, geometrical and developmental parameters, in a range corresponding a given species, found in literature.

In [4]:
nb_phy=15 # number of phytomers on the main stem

# Stem
height=1.3*max(height_canopy) # potential plant height
stem_q=1.1 # parameter for ligule height distribution along axis
diam_base=2.5 # stem base diameter
diam_top=1.5 # stem top diameter

# Leaf blade area 
leaf_area=1.3*max(leaf_area_plant) # potential plant leaf area
rmax=0.8 # parameter for leaf area distribution along axis
skew=0.0005 # parameter for leaf area distribution along axis
wl=0.12 # leaf blade width-to-length ratio 
klig=0.6 # parameter for leaf blade shape
swmax=0.55 # parameter for leaf blade shape
f1=0.64 # parameter for leaf blade shape
f2=0.92 # parameter for leaf blade shape

# Leaf blade position in space
insertion_angle=35 # leaf blade insertion angle
scurv=0.7 # leaf blade relative inflexion point
curvature=120 # leaf blade insertion-to-tip angle
phyllotactic_angle=180 # phyllotactic angle
phyllotactic_deviation=10 # half-deviation to phyllotactic angle

# Development
phyllochron=30 # phyllochron, i.e. stem element appearance rate
plastochron=20 # plastochron, i.e. leaf blade appearance rate

# Senescence 
leaf_lifespan = 300 # leaf lifespan from appearance

# Tillering
nb_tillers=10 # number of tillers
tiller_delay=1 # delay, as factor of phyllochron, between the appearance of a phytomer and the appearance of its tiller
reduction_factor=1 # reduction factor between tillers of consecutive order

## 4. Instantiate ArchiCrop object

In [5]:
plant = ArchiCrop(height=height, 
                  nb_phy=nb_phy,
                  leaf_area=leaf_area,
                  wl=wl, diam_base=diam_base, diam_top=diam_top, 
                  insertion_angle=insertion_angle, scurv=scurv, curvature=curvature, 
                  klig=klig, swmax=swmax, f1=f1, f2=f2, 
                  stem_q=stem_q, rmax=rmax, skew=skew,
                  phyllotactic_angle=phyllotactic_angle,
                  phyllotactic_deviation=phyllotactic_deviation,
                  phyllochron=phyllochron, 
                  plastochron=plastochron, 
                  leaf_lifespan=leaf_lifespan,
                  nb_tillers=nb_tillers, tiller_delay=tiller_delay, reduction_factor=reduction_factor,
                  daily_dynamics=daily_dynamics)

## 5. Generate a potential plant 

Generate a potential plant from architectural parameters. 

This creates an MTG (Multi-scale Tree Graph, cf [doc](https://mtg.readthedocs.io/en/latest/user/tutorial.html)) as a property of the object plant.

In [6]:
plant.generate_potential_plant()

In [7]:
g = plant.g
scale = g.max_scale()-1
print(g.property_names())
oawidgets.mtg.plot(g)

['edge_type', 'label', 'rank', 'mature_length', 'length', 'visible_length', 'is_green', 'mature_stem_diameter', 'stem_diameter', 'azimuth', 'grow', 'age', 'stem_lengths', 'start_tt', 'end_tt', 'shape', 'leaf_area', 'visible_leaf_area', 'senescent_area', 'senescent_length', 'wl', 'tck', 'srb', 'srt', 'lrolled', 'd_rolled', 'shape_max_width', 'dead', 'leaf_lengths', 'senescent_lengths', 'senescence', 'geometry', 'anchor_point', 'color']
Local cdn resources have problems on chrome/safari when used in jupyter-notebook. 


## 6. Simulate growth and senescence of the plant

Simulate growth and senescence of this plant following the daily dynamics.

In [8]:
growing_plant = plant.grow_plant()

## 7. Plot 3D scene

In [9]:
times = [t for i,t in enumerate(thermal_time)]
mean_time = sum(times) / len(times)
positions = [ (0, 1*(t-mean_time), 0) for t in times]
nice_green = Color3((50, 100, 0))
scene, _ = build_scene([g for i,g in enumerate(list(growing_plant.values()))], position=positions, senescence=True, leaf_material = Material(nice_green), stem_material=Material(nice_green))
PlantGL(scene)

Plot(antialias=3, axes=['x', 'y', 'z'], axes_helper=1.0, axes_helper_colors=[16711680, 65280, 255], background…